In [ ]:
# https://huggingface.co/CompVis/stable-diffusion-v1-4
# https://github.com/huggingface/diffusers
# first you need to log in on the console: "huggingface-cli login"

In [ ]:
import datetime
import os
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline

import requests
from PIL import Image
from io import BytesIO

torch.cuda.empty_cache()
torch.cuda.is_available()
os.makedirs("generatedImages/", exist_ok=True)
os.makedirs("generatedImages/3. img2img", exist_ok=True)

In [ ]:
model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda"

In [ ]:
# use this
# pipe = StableDiffusionPipeline.from_pretrained(model_id, use_auth_token=True)

In [ ]:
# use this if less than 10 GB GPU RAM is available
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, revision="fp16", use_auth_token=True)
#pipe = StableDiffusionPipeline.from_pretrained(model_id, use_auth_token=True)

In [ ]:
pipe = pipe.to(device)
urlOrImagename = ""

# Could probably use an inline lambda for this
def dummy(images, **kwargs):
    return images, False
pipe.safety_checker = dummy     # Disable safety check
#pipe.enable_attention_slicing() #reduces GPU RAM

In [ ]:
# image from url
urlOrImagename = "https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg"
response = requests.get(urlOrImagename)
init_image = Image.open(BytesIO(response.content)).convert("RGB")
init_image = init_image.resize((768, 512))

In [ ]:
# or image by path
imgFileName = "sketch-mountains-input.jpg"
urlOrImagename = "generatedImages/3. img2img/1. Src/" + imgFileName
init_image = Image.open(urlOrImagename).convert("RGB")

In [ ]:
display(init_image)

In [ ]:
prompt = "A fantasy landscape, trending on artstation"
folderName = "A fantasy landscape, trending on artstation"

foldertimestamp = str(int(datetime.datetime.now().timestamp()* 100000))
folderName = foldertimestamp + "_" + folderName

guidanceScale = 7.5
numberOfSteps = 50
os.makedirs("generatedImages/3. img2img/2. result", exist_ok=True)
os.makedirs("generatedImages/3. img2img/2. result/" + folderName, exist_ok=True)

with open("generatedImages/3. img2img/2. result/" + folderName + "/" + "prompt.txt", 'w') as f:
    f.write("Image2Image")
    f.write("\n  urlOrImagename: " + urlOrImagename)
    f.write("\n" + prompt)
    f.write("\n" + "guidanceScale: " + str(guidanceScale))
    f.write("\n" + "numberOfSteps: " + str(numberOfSteps))

for i in range(10):
    print(i)
    with autocast("cuda"):
        image = pipe(prompt, init_image=init_image, guidance_scale=guidanceScale, num_inference_steps=numberOfSteps)["sample"][0];
    image.save("generatedImages/3. img2img/2. result/" + folderName + "/" + str(int(datetime.datetime.now().timestamp()* 100000)) + ".png")